# Medway Polling station locations

In [ ]:
# https://www.medway.gov.uk/downloads/file/8737/polling_station_locations_for_general_election_2024

In [ ]:
# !pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import requests
import time
import json
from glob import glob
import math

In [ ]:
def get_coordinate_from_file(row):
    # print(row)
    address = json.load(open(f"data/address_to_coord/{row['STATION NO.']}.json"))
    if len(address) > 0:
        row['longitude'] = math.ceil((float(address[0]['lon']) * 1e6)) / 1e6
        row['latitude'] =  math.ceil((float(address[0]['lat']) * 1e6)) / 1e6
    return row

df_polling_stations = (
    pd.concat([
        pd.read_excel('data/Polling_station_locations.xlsx', skiprows=0, header=3, nrows=39).assign(**{'constituancy': 'Chatham and Aylesford'}),
        pd.read_excel('data/Polling_station_locations.xlsx', header=46, nrows=29).assign(**{'constituancy': 'Gillingham and Rainham'}),
        pd.read_excel('data/Polling_station_locations.xlsx', header=79, nrows=46).assign(**{'constituancy': 'Rochester and Strood'})
    ], axis=0)
    .assign(**{'POLLING STATION': lambda _df: _df["POLLING STATION"].ffill()})
    .apply(get_coordinate_from_file, axis=1)
)
    
        

df_polling_stations.to_csv('data/polling_stations_2024.csv', index=False)

In [ ]:
"""TODO: GET API KEY FROM SECRETS FILE"""


def get_address(row):
    
    path_address = f"data/address_to_coord/{row['STATION NO.']}.json"
    print(path_address)
    if os.path.exists(path_address):
        print('Address already exists - skipping file')
    else:        
        # Try dropping first part of address
        address = ','.join(row['POLLING STATION'].split(',')[4:]).strip()
        
        url = f"https://geocode.maps.co/search?q={address}&api_key={api_key}"        
        result = requests.get(url)
        
        print(result.json())
            
        # Cache data
        with open(path_address, 'w', encoding='utf-8') as f:
            json.dump(result.json(), f)
        
        # raise KeyboardInterrupt

        
        time.sleep(3)


df_polling_stations.apply(get_address, axis=1)

In [ ]:
# Delete emptyp address files
def delete_empty_address_files():
    for path_file in glob('data/address_to_coord/*.json'):
        address = json.load(open(path_file))

        if len(address) == 0:
            print(f'removing {path_file}')
            os.remove(path_file)
    
delete_empty_address_files()
